In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
from torch.utils.data import DataLoader, TensorDataset, Dataset
import numpy as np
import pandas as pd
import metal
import os
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM

Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


In [3]:
from dataset import STSBDataset

In [4]:
from BERT_tasks import create_task
tasks = [create_task('STS-B')]

100%|██████████| 1500/1500 [00:01<00:00, 1351.09it/s]


In [ ]:
from metal.end_model import EndModel
from metal.mmtl.metal_model import MetalModel
from metal.mmtl.trainer import MultitaskTrainer

model = MetalModel(tasks, verbose=False)
trainer = MultitaskTrainer()
trainer.train_model(
    model,
    tasks,
    n_epochs=4, 
    lr=0.005, 
    progress_bar=True,
    log_valid_metrics=["foo_task/accuracy", "foo_task/precision"],
    checkpoint_metric="foo_task/accuracy",
)